# Word2Vec

In [ ]:
#updating gensim and numpy to run Word2Vec library function
!pip install --upgrade gensim
!pip3 install numpy==1.23.5

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download("popular")
import re

import string
import matplotlib.pyplot as plt


from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.decomposition import PCA

# constants
sw = stopwords.words('english')
#adding more stopwords w.r.t. the data
sw.extend(['http', 'https', 'get', 'amp', 'co', 'need', 'like', 'go', 'coronavirus', 'time', 'please','going' ])
sw.sort()

# constants
PATH = 'Corona_NLP_test.txt'
plt.style.use('ggplot')

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category = DataConversionWarning)


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [ ]:
# import data
lines = []
with open(PATH, 'r') as f:
    for l in f:
        lines.append(l)

#lines['text']=lines['text'].astype(str)

# remove new lines
lines = [line.rstrip('\n') for line in lines]

# make all characters lower
lines = [line.lower() for line in lines]

# remove punctuations from each line
lines = [line.translate(str.maketrans('', '', string.punctuation)) for line in lines]

# tokenize
lines = [word_tokenize(line) for line in lines]


In [ ]:
def remove_stopwords(lines, sw = sw):
    '''
    The purpose of this function is to remove stopwords from a given array of
    lines.

    params:
        lines (Array / List) : The list of lines you want to remove the stopwords from
        sw (Set) : The set of stopwords you want to remove

    example:
        lines = remove_stopwords(lines = lines, sw = sw)
    '''

    res = []
    for line in lines:
        original = line
        line = [w for w in line if w not in sw]
        if len(line) < 1:
            line = original
        res.append(line)
    return res

filtered_lines = remove_stopwords(lines = lines, sw = sw)
#filtered_lines

In [ ]:
from gensim.models import Word2Vec as w2v

w = w2v(
    filtered_lines,
    min_count=3,
    sg = 1,
    window=7
)

#print(w.wv.most_similar('three'))

emb_df = (
    pd.DataFrame(
        [w.wv.get_vector(str(n)) for n in w.wv.key_to_index],
        index = w.wv.key_to_index
    )
)
print(emb_df.shape)
#emb_df.head(50)

(3521, 100)


# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


#from nltk.corpus import stopwords
#from nltk import word_tokenize
#from sklearn.decomposition import PCA
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

plt.style.use('ggplot')

# constants
PATH = 'Corona_NLP_test.txt'

plt.style.use('ggplot')
# nltk.download('punkt')
# nltk.download('stopwords')

#lines = pd.read_excel("Corona_NLP_test.xlsx")
#lines['Text']=lines['Text'].astype(str)


# import data
lines = []
with open(PATH, 'r') as f:
    for l in f:
        lines.append(l)




In [ ]:
# Cleaning the texts

ps = PorterStemmer()
wordnet=WordNetLemmatizer()
corpus = []
for i in range(len(lines)):
    review = re.sub('[^a-zA-Z]', ' ', lines[i])
    review = review.lower()
    review = review.split()
    review = [wordnet.lemmatize(word) for word in review if not word in set(sw)]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
# Creating the TF-IDF model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
cv = TfidfVectorizer(ngram_range=(1,1))
corpus1 = cv.fit_transform(corpus)
avg = corpus1.mean(axis=0)
avg = pd.DataFrame(avg, columns=sorted(cv.vocabulary_.keys()))
avg=avg.T
avg=avg.rename(columns={0:'score'})
avg['word']=avg.index
avg=avg.sort_values('score', ascending=False)
avg=avg[:50]


In [ ]:
cv_bigram = TfidfVectorizer(ngram_range=(3,3))
corpus1_bigram= cv_bigram.fit_transform(corpus)
avg_bigram=corpus1_bigram.mean(axis=0)
avg_bigram= pd.DataFrame(avg_bigram, columns=sorted(cv_bigram.vocabulary_.keys()))
avg_bigram=avg_bigram.T
avg_bigram=avg_bigram.rename(columns={0:'score'})
avg_bigram['word']=avg_bigram.index
avg_bigram=avg_bigram.sort_values('score', ascending=False)

In [ ]:
unigrams_list=avg['word'].tolist()
bigrams_list=avg_bigram['word'].tolist()

In [ ]:
def convert(lst):
  return ([item.split() for item in lst])

In [ ]:
bigrams_split=convert(bigrams_list)

In [ ]:
check =pd.DataFrame(columns=['topic','subtopic'])

In [ ]:
for i in unigrams_list:
  counter =0
  for j in bigrams_split:
    if counter<5 and (i==j[0] or i==j[1] or i==j[2]):
      bigram_words=' '.join(j)
      check = pd.concat([check, pd.concat([pd.Series(i,name='topic'),pd.Series(bigram_words,name='subtopic')],axis=1)], axis=0)
      counter = counter+1

# Value of Constraint a

In [ ]:
ans = avg.head(15)
print(ans)
""""
Here, we are manipulating the tf-idf values to fit the bounds of Henon Map by multiplying it with (100)
"""
keywords_values = ans['score']*(100)
word_index = ans['word']
word_index = word_index.to_numpy()
constraint_a = keywords_values.to_numpy()
#constraint_a



                        score                 word
covid                0.037539                covid
food                 0.015422                 food
store                0.013970                store
grocery              0.011254              grocery
stock                0.010669                stock
people               0.009588               people
supermarket          0.008454          supermarket
shopping             0.007522             shopping
panic                0.007339                panic
online               0.007298               online
price                0.006974                price
toilet               0.005804               toilet
paper                0.005757                paper
coronaviruspandemic  0.005400  coronaviruspandemic
buying               0.004634               buying


# Value of Constraint b

In [ ]:
constraint_b = []

for word in ans['word']:
  sum=0
  temp = w.wv.most_similar(word, topn = 100)
  df_temp=pd.DataFrame(temp, columns=['word','score'])
  #print(df_temp)
  for value in df_temp['score']:
    sum=sum+value
  sum=sum/300
  constraint_b.append(sum)
  #if(len(word)>14):
    #print(word,"\t", sum, end ="\n")
  #elif(len(word)>6):
    #print(word,"\t\t", sum, end ="\n")
  #else:
    #print(word,"\t\t\t", sum, end ="\n")



# Lyapunov Exponent

In [ ]:
from numpy import *


class HenonMap:
    """ Class to hold the parameters of the Henon Map and evaluate it along its directional derivative.
        The default parameters are chosen as the canonical ones in the initialization.
        It instantiates a callable object f_df, in such a way that f_df(xy, w)
        returns two values f(xy) and df(xy, w), where
        f(xy) is the Henon map evaluated at the point xy and
        df(xy, w) is the differential of the Henon evaluated at xy in the direction of w.
    """
    def __init__(_, a, b):
        _.a, _.b = a, b

    def f(_, xy):
        x, y = xy
        return array([_.a - x ** 2 + _.b * y, x])

    def df(_, xy, w):
        x, y = xy
        j = array([[-2 * x, _.b],
                   [1, 0]])
        return j @ w

    def __call__(_, xy, w):
        return _.f(xy), _.df(xy, w)

In [ ]:
from numpy import *
from numpy.linalg import *


def lyapunov_exponent(f_df, single_initial_condition=None, multiple_initial_conditions=None,
                      tol=0.01, max_it=1000, min_it_percentage=0.1):
    """Numerical approximation of all Lyapunov Exponents of a map.
    Ref: Alligood, Kathleen T., Tim D. Sauer, and James A. Yorke. Chaos. Springer New York, 1996.
    Parameters
    ----------
    f_df : callable
        The map to be considered for computation of Lyapunov Exponents
        f_df (x, w) -> (f(x), df(x, w)), where
        f(x) is the map evaluated at the point x and
        df(x, w) is the differential of this map evaluated at x in the direction of w.
    single_initial_condition : ndarray of shape (n)
        Single initial condition to computed the Lyapunov Exponent.
    multiple_initial_conditions : ndarray of shape (m,n)
        m initial conditions to computed the average Lyapunov Exponent.
    tol : float
        Tolerance to stop the approximation.
    max_it : int
        Max numbers of iterations.
    min_it_percentage : float, optional
        Min number of iterations as a percentage of the max_it.
    Returns
    -------
    : ndarray of shape (n)
        The Lyapunov exponents computed associated to a single initial condition or
        the average value considering several initial conditions
    """

    if multiple_initial_conditions is not None:
        (m, n) = shape(multiple_initial_conditions)
        ls = zeros((m, n))
        for i in range(m):
            ls[i] = lyapunov_exponent(f_df=f_df, single_initial_condition=multiple_initial_conditions[i],
                                      tol=tol, max_it=max_it, min_it_percentage=min_it_percentage)
        return apply_along_axis(lambda v: mean(v), 0, ls)

    elif single_initial_condition is None:
        raise Exception('Either single_initial_condition or multiple_initial_conditions must be provided.')

    n = len(single_initial_condition)
    x = single_initial_condition
    w = eye(n)
    h = zeros(n)
    trans_it = int(max_it * min_it_percentage)
    l = -1

    for i in range(0, max_it):
        x_next, w_next = f_df(x, w)
        w_next = orthogonalize_columns(w_next)

        h_next = h + log_of_the_norm_of_the_columns(w_next)
        l_next = h_next / (i + 1)

        if i > trans_it and norm(l_next - l) < tol:
            return sort(l_next)

        h = h_next
        x = x_next
        w = normalize_columns(w_next)
        l = l_next

    return (-1,-1)
    raise Exception('Lyapunov Exponents computation did no convergence' +
                    ' at ' + str(single_initial_condition) +
                    ' with tol=' + str(tol) +
                    ' max_it=' + str(max_it) +
                    ' min_it_percentage=' + str(min_it_percentage))


def orthogonalize_columns(a):
    q, r = qr(a)
    return q @ diag(r.diagonal())


def normalize_columns(a):
    return apply_along_axis(lambda v: v / norm(v), 0, a)


def log_of_the_norm_of_the_columns(a):
    return apply_along_axis(lambda v: log(norm(v)), 0, a)

In [ ]:
import unittest
from numpy import *
from numpy.linalg import *

#HENON_MAP_REFERENCE_SOLUTION = array([-1.61, 0.41])
#TEST_TOLERANCE = 0.01
x_points, y_points = mgrid[-1:1:0.25, -1:1:0.25]
HENON_MAP_INITIAL_CONDITIONS = column_stack((x_points.ravel(), y_points.ravel()))


# Ref: Alligood, Kathleen T., Tim D. Sauer, and James A. Yorke. Chaos. Springer New York, 1996.
#range bound = (1.1,0.4), (1.8,0.1); always chaotic at (1.4,0.3)
henon_values = []
for i in range(0,15):
  HENON_MAP = HenonMap(constraint_a[i], constraint_b[i])
  a =  lyapunov_exponent(HENON_MAP, multiple_initial_conditions=HENON_MAP_INITIAL_CONDITIONS)
  print(i, ',',a)
  henon_values.append(a)


class LyapunovExponentsTestCase(unittest.TestCase):
    def henon_map_test(_):
        l = lyapunov_exponent(HENON_MAP, multiple_initial_conditions=HENON_MAP_INITIAL_CONDITIONS)
        _.assertEqual(norm(HENON_MAP_REFERENCE_SOLUTION - l, inf) < TEST_TOLERANCE, True)


if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

<ipython-input-39-a07c66d0d108>:84: RuntimeWarning: divide by zero encountered in log
  return apply_along_axis(lambda v: log(norm(v)), 0, a)
<ipython-input-39-a07c66d0d108>:80: RuntimeWarning: invalid value encountered in divide
  return apply_along_axis(lambda v: v / norm(v), 0, a)
<ipython-input-38-583cf6318a15>:17: RuntimeWarning: overflow encountered in double_scalars
  return array([_.a - x ** 2 + _.b * y, x])
<ipython-input-38-583cf6318a15>:23: RuntimeWarning: invalid value encountered in matmul
  return j @ w
<ipython-input-39-a07c66d0d108>:80: RuntimeWarning: divide by zero encountered in divide
  return apply_along_axis(lambda v: v / norm(v), 0, a)


0 , [-1. -1.]
1 , [-1. -1.]
2 , [-1.68416455  0.37497413]
3 , [-1.4835986   0.19575036]
4 , [-1.29720292  0.07176951]
5 , [-0.7156278  -0.43114817]
6 , [-1.02988252 -0.16175524]
7 , [-0.73261749 -0.59907005]
8 , [-0.67679417 -0.54517721]
9 , [-0.76381987 -0.61995599]
10 , [-0.61865679 -0.50783496]
11 , [-0.72383802 -0.55228756]
12 , [-0.72279111 -0.54817596]
13 , [-0.91520318 -0.42517845]



----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


14 , [-1.02557867 -0.20209051]


In [ ]:
#print(henon_values)
for i in range(15):
  if(len(ans['word'][i])>14):
    print(ans['word'][i], "\t", henon_values[i], end = "\n")
  elif(len(ans['word'][i])>6):
    print(ans['word'][i], "\t\t", henon_values[i], end = "\n")
  else:
    print(ans['word'][i], "\t\t\t", henon_values[i], end = "\n")


covid 			 [-1. -1.]
food 			 [-1. -1.]
store 			 [-1.68416455  0.37497413]
grocery 		 [-1.4835986   0.19575036]
stock 			 [-1.29720292  0.07176951]
people 			 [-0.7156278  -0.43114817]
supermarket 		 [-1.02988252 -0.16175524]
shopping 		 [-0.73261749 -0.59907005]
panic 			 [-0.67679417 -0.54517721]
online 			 [-0.76381987 -0.61995599]
price 			 [-0.61865679 -0.50783496]
toilet 			 [-0.72383802 -0.55228756]
paper 			 [-0.72279111 -0.54817596]
coronaviruspandemic 	 [-0.91520318 -0.42517845]
buying 			 [-1.02557867 -0.20209051]


# Chaotic and Non-chaotic words


In [ ]:
chaotic_words=[]
non_chaotic_words =[]
for i in range(15):
  if(henon_values[i][1]>0):
    chaotic_words.append(i)
  else:
    non_chaotic_words.append(i)

print("Chaotic Words:\n")
for w in chaotic_words:
  a = ans.loc[word_index[w]]
  print(a['word'])

print("\n\nNon-Chaotic Words:\n")
for w in non_chaotic_words:
  a = ans.loc[word_index[w]]
  print(a['word'])


Chaotic Words:

store
grocery
stock


Non-Chaotic Words:

covid
food
people
supermarket
shopping
panic
online
price
toilet
paper
coronaviruspandemic
buying
